# 3 监督学习
监督学习的目标是通过构建自变量与因变量间关系的拟合模型，解释数据的规律性，以便预期未来的结果。

即：当 $y$ 已知时，求解功能函数 $F$ ，使 $y = F(x)$ ；现实情况中，该公式多无法绝对相等，只能尽可能拟合$y$中的绝大多数。

根据 $y$ 的差异划分为：
- **回归**（$y$ 无限集）
- **分类**（$y$ 有限集）

其中，分类问题常见的是二分类问题（比如垃圾邮件识别），但也处理拥有任意数量类的多分类问题；前者模型称作二分类器，后者称作多类分类器（多项式分类器）。

处理多分类问题时，如随机森林、朴素贝叶斯等可直接支持；而类似SVM、线性分类器等严格的二分类器，可使用如下策略：
- **一对所有（OvA）**：创建与类别数目相等的$N$个分类器，单个分类器训练时，仅选取单一类别为正例其余均为负例，决策时选取决策分数最高的分类器所对应的类。
- **一对一（OvO）**：将类别数量$N$两两配对作为训练子集单独训练二分类器，共计训练$\frac{N*(N-1)}{2}$个分类器，决策时选出全部分类器中胜出最多的类（比如投票机制）。

根据 $F$ 的差异划分为：
- **参数模型**：预设数据服从某个分布（$F$），目标是评估数据分布的参数值，如线性回归、逻辑回归、感知机等
- **非参数模型**：预设数据分布存在但未知（参数可变且无数量上限），只能通过非参数统计的方法进行推断，如knn、决策树、朴素贝叶斯、支持向量机、神经网络等

此外，实际应用中，多数监督学习算法都支持处理回归&分类问题。

## 3.1 参数模型
如前所述，模型最重要的三要素：模型假设、损失函数、寻解算法；此处，我们将模型统一抽象为$F(X)$，聚焦探讨损失函数与寻解算法。

### 3.1.1 回归问题的损失函数
损失函数，表达了建模者希望模型达成的目标。针对回归问题，我们必然希望模型的预测值 $\hat y$ 与真值 $y$ 越接近越好（最佳情况一摸一样），即：$y - \hat y$ 越小越好。

如前所述模型要兼顾偏差和方差，所以应该将训练数据（假设$n$条记录）都被很好的拟合，故优化如下：
$$ MBE(平均偏差误差) = \frac{1}{n}\sum_{i = 1}^n(y_i - \hat y_i) $$
从公式中不难发现，MBE会受到误差方向的影响，产生正负相互抵消，评估损失的精确度不足的问题，改良如下：
$$ MAE(平均绝对值误差,L1Loss) = \frac{1}{n}\sum_{i = 1}^n|y_i - \hat y_i| $$
MAE虽然改善了正负偏差相互抵消的问题，但是函数在零点不可导，为后续寻找全局最优解造成了困扰，故再次优化：
$$ RMSE(均方根误差,L2Loss) = \sqrt {MSE(均方误差)} = \sqrt {\frac{1}{n}\sum_{i = 1}^n(y_i - \hat y_i)^2} $$
RMSE具有处处可导的特性，相比于内层的MSE，数值量级下降很多，使其成为了最常使用的回归问题损失函数。值得注意的是，函数的指数特性对于异常点（真值与预测值偏差极大的）敏感，如存在异常点，会牺牲正常点的参数优化，着重于优化对损失函数贡献更高的离群点，所以数据预处理时要多加注意。

下面介绍一些改良的损失函数：
$$ HuberLoss(平滑绝对误差) = \sum_{i=1}^n \begin{cases} 0.5*(y_i - \hat y_i)^2 & |y_i - \hat y_i|<\delta \\ \delta * (|y_i - \hat y_i|-0.5 \delta ^2) & else \end{cases}$$
当预测值与真值差别过大（大于 $\delta$ ）时，使用L1范数，其余情况使用L2范数，可平衡异常点对损失函数值的影响，该函数最常用的形态是：$ smooth_{L1}(\delta = 1) = \sum_{i=1}^n \begin{cases} 0.5*(y_i - \hat y_i)^2 & |y_i - \hat y_i|<1 \\ |y_i - \hat y_i|-0.5 & else \end{cases} $

$$ LogCoshLoss = \sum_{i=1}^n log(cosh(y_i - \hat y_i)) $$
对于较小的 $x$，$log(cosh(x))$ 近似于 $\frac{1}{2}x^2$，对于较大的 $x$，近似于$|x|-log(2)$，具有Huber loss所有的优点且二阶导数处处可微；但当误差极大时，一阶导数和Hessian会变成定值，导致如XGBoost出现缺少分裂点的情况。
$$ QuantileLoss(分位数损失) = \sum_{y_i<\hat y_i}(1-\lambda)|y_i-\hat y_i| + \sum_{y_i\geq\hat y_i}\lambda|y_i - \hat y_i|, \lambda \in [0,1] $$
适用于偏重对某区间进行更严苛拟合需求的现实问题，通过基于对正反误差的重视程度，调节$\lambda$（分位值）给予不同的惩罚，让函数对中值产生偏移，突出重点区间。

以上只是我熟识的回归问题损失函数，当然还有许多未提及，亦可根据实际问题自定义。

#### 3.1.2 分类问题的损失函数
对于分类问题，使用RMSE及其相关变体理论也是可以的，但是部分模型叠加了符号函数，可能造成损失函数密集的局部最优解，干扰全局最优解的寻找；故下面阐述比较适合分类问题的损失函数。
#### 3.1.2.1 负似然对数
负似然对数损失函数就是**似然函数**的负值，那什么是似然函数呢？用二分类问题进行假设说明：
- 事件有且仅有互斥的正、反两种结果（比如垃圾邮件分类），正例记为1，负例记为0。
- 事件正例发生的概率为：$p$，则事件反例概率为：$1-p$。
- 对事件累计实验$N$次（或收集到$N$条记录），其中正例$m$次，反例$n$次。

依据假设，$m$次实验的似然函数为：$L(p)=p^m*(1-p)^n$；如果不是实验，而是单纯的数据记录，考虑无序性，似然函数更新为：$L(p)=C^m_N*p^m*(1-p)^n$）；由于固定系数并不改变函数单调性，故着眼于：$L(p)=p^m*(1-p)^n$。依据统计学中频率学派的观点，充分多的已发生事件集可能性最高，即 $L(p)$ 取最大值的 $p$ 便是真实的事件正例概率。将 $p$ 替换为模型函数的预测结果：$h_\theta(x)$，可获得公式：$ p(\theta) = \begin{cases} h_\theta(x) & y=1 \\ 1-h_\theta(x) & y=0 \end{cases} = h_\theta(x)^y + (1-h_\theta(x))^{(1-y)} $，其中 $y$ 为真实正例或者负例标签。将 $p(\theta)$ 代入似然函数公式：
$$ L(\theta) = \prod_{i=1}^n h_\theta(x)^{y_i} + (1-h_\theta(x))^{(1-y_i)}  $$

这样就将求解函数参数的问题等价于寻找合适的 $ \theta $ 使似然函数取最大值，则负似然函数就是求解最小值，可作损失函数：
$$ cost(\theta) = -L(\theta) = - \prod_{i=1}^n h_\theta(x)^{y_i} + (1-h_\theta(x))^{(1-y_i)} $$

该损失函数的求导颇为复杂，而对 $ L(\theta) $ 取对数并不影响单调性，最小值的取值点不变：
$$ cost(\theta) = -lnL(\theta) = - \sum_{i=1}^n y_i * ln(h_\theta(x)) + (1-y_i)*ln(1-h_\theta(x)) $$

矩阵形式为：
$$ cost(\theta) = -Y^T * ln(h_\theta(X))-(E - Y)^T * (E - ln(h_\theta(X))) $$

#### 3.1.2.2 交叉信息熵
因个人水平不足以理论化介绍交叉信息熵(cross entropy)，故下面用粗浅的认知进行阐述，准确的信息可查阅信息论相关文档。

**信息**，可以说是一种价值度量。以新闻事件来类比，超出人们想象的、小概率发生的事件我们才会将其作为头条新闻，因为该新闻的价值高；类似的，一个系统中，小概率事件蕴含的信息值高。怎么度量呢？因为信息起源于计算机对信息的记录，只有0/1两种标记方案，故以二分类决策推测某事件的具体结果为类比，最优策略总是优先选择最可能发生的结果，此后逐层递减。这个准则下，确定某个事件结果发生所消耗的路径长度就是信息，计算公式：$log_2(\frac{1}{p})$，$p$ 即为某事件的概率（此处我认为$log_n(\frac{1}{p})$,$n$为分叉数）。

**信息熵**：分类系统中多个类别信息值的加权和：$ \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $。

**交叉熵**：基于模型预测的事件概率分布$q_i$的信息加权和：$ \sum_{i=1}^n p_i * log_2(\frac{1}{q_i}) $。

**相对熵**：$ 相对熵 = 交叉熵 - 信息熵 = \sum_{i=1}^n p_i * log_2(\frac{p_i}{q_i})$

所以损失函数可以使用相对熵，目标是使相对熵最小，$ cost(\theta) = \sum_{i=1}^n p_i * log_2(\frac{1}{q_i}) - \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $，由于信息熵是常量，故损失函数可以精简为：$ cost(\theta) = \sum_{i=1}^n p_i * log_2(\frac{1}{q_i})= -\sum_{i=1}^n p_i * log_2(q_i) $

令$q_i$为模型$h_\theta(x)$预测结果,则损失函数为：
$$ cost(\theta) = -\sum_{i=1}^n p_i * log_2(q_i) = -\sum_{i=1}^n p_i * [y_i * log_2(h_\theta(x)) + (1-y_i) * log_2(1 - h_\theta(x))] $$

#### 3.1.2.3 指数损失(exponential loss)
*暂时还没研究透彻，adaboost的损失函数*

#### 3.1.2.4 Hinger损失
*暂时还没研究*

#### 3.1.2.5 感知损失（Hinger损失的特例）
*暂时还没研究*

#### 3.1.2.6 铰链损失
*暂时还没研究*

### 3.1.3 寻解算法：最小二乘法（OLS）
**最小二乘法（最小平方法）**：通过最小化RMSE，寻找模型的最佳匹配参数；求解最小二乘法结果的方法有**矩阵解法**和**梯度下降法**。

*以前我以为最小二乘法和梯度下降法是两个方法，其实梯度下降法只是最小二乘法求解的方法之一。*

#### 3.1.3.1 矩阵解法
为方便后续讲解，设定前提假设如下：
- **模型假设**：$ F(x)=a_1*x_1+a_2*x_2+……+a_n*x_n+b=[x_1,x_2,……,x_n,1] \cdot [a_1,a_2,……,a_n,b]^T=XA $
- **损失函数**：$ L(x,a)=\frac{1}{2n}\sum_{i=1}^n[(a_1*x_{i1}+a_2*x_{i2}+……+a_n*x_{in}+b)-y_i]^2=\frac{1}{2n}(XA-Y)^T(XA-Y) $

依据假设，当损失函数的一阶偏导数（$\partial A$）为零时，损失函数取最小值，即：
$$ \frac{\partial L}{\partial A}=\frac{\partial L}{\partial a_1}+\frac{\partial L}{\partial a_2}+……+\frac{\partial L}{\partial a_n}=\frac{1}{n}\sum_{i=1}^n(a_1x_{i1}+a_2x_{i2}+……+a_nx_{in})*(x_{i1}+x_{i2}+……+x_{in})=\frac{X^T(XA-Y)}{n}=0 $$
求解：
$$ A = (X^TX)^{-1}X^TY $$
以上的推导过程表明矩阵解法有如下限制：
- 1.系数矩阵$A$必须存在逆矩阵（舍弃边缘特征）。
- 2.系数矩阵$A$的求逆过程的耗时受矩阵规模影响（降维处理）。
- 3.模型假设若非线性，求解将十分复杂或难以开展（转化线性关系）。

*（）中是一些预处理方案，优化矩阵解法的适配问题。*

#### 3.1.3.2 梯度下降法
梯度下降法是利用梯度逐步迭代模型参数，使损失函数达到最小值的方法。
```
想象你和贝爷一起进行丛林探险，露宿山崖边缘，晨间丛林里雾气弥漫，你们选择下至山下的峡谷补充水源。此时，你们**各选了一个位置**开始下山，你用脚一点点感受被踩踏的岩石，选择合适的落脚点，你总是**优先选择垂直向下（即坡度最陡）**的方向下山，重复如此，最终你们下到了山脚。
```
梯度下降法的过程与之相似，使用相同的假设表达如下：
- **模型假设**：$ F(x)=XA $ 并随机初始化$A$中参数。
- **损失函数**：$ L(x,a)=\frac{1}{2n}(XA-Y)^T(XA-Y) $。
- **偏导数**：$ \frac{\partial L}{\partial A}=\frac{1}{n}X^T(XA-Y) $。
- **梯度下降**：$ A=A-\alpha \frac{\partial L}{\partial A},\alpha \in (0,1) $。
- **终止条件**：损失不可改善或达预设终止条件时。

学习率（$\alpha$）与梯度（$\frac{\partial L}{\partial A}$）共同决定每轮参数的迭代步长，其中学习率是用户设定值，需注意：
- 模型训练前应特征缩，提高 $\alpha$ 作用效果。
- $\alpha$ 设定过小，耗时严重；设定过大，反复跳跃难收敛。

与矩阵解法中求解一阶导数为零不同，由于训练数据 $X$，随机参数 $A$、学习率 $\alpha$ 均为已知项，其目标是将现有的随机参数不断改善成最优函数的接近值，所以突破了矩阵解法的限制，更加通用。但梯度下降法的主要挑战是损失函数的选择，有些损失函数除了全局最小值外，还存在多个局部最小值，有时可能永远到不了全局最小值。

实际使用梯度下降法时，由于计算梯度方式的差异，衍生出三种方法：
- **批梯度下降（BGD）**：每轮迭代计算梯度时使用全部训练数据；优点是朝着最小值迭代，缺点是样本值很大时每轮更新速度会很慢。
- **随机梯度下降（SGD）**：每轮迭代计算梯度时随机抽取一条训练数据；优点是每轮更新速度快，缺点是样本噪点较多时未必朝着极小值方向更新（多轮迭代能保障整体大致朝着极小值方向更新）。
- **小批量梯度下降（MBGD）**：每轮迭代计算梯度时随机抽取一定比例训练数据；优点是平衡更新梯度方向准确性与更新速度，缺点是batch值较难确定。

随机性可以很好的跳过局部最优值，所以比较推荐使用小批量梯度下降法；对于随机梯度下降较难快速达到最小值问题，可使用逐渐降低学习率改善，这个过程被称为模拟退火。

上述虽然只使用了简单的回归问题举例说明，但是分类问题也适用（负似然对数与交叉信息熵的梯度形式：$X^T*(h_\theta(x)-Y)$），亦可推广到更高维，更复杂的损失函数中使用。

## 3.2 非参数模型
非参只是数据分布不可知，不代表建模本身没有参数需要调整，我们还是要通过控制推断参数，进而控制模型的拟合情况。

### 3.2.1 knn
knn是一个简单实用、基于实例学习的算法，其原理是：

1. 以选定的建模特征构建多维空间
2. 计算预测数据与样本数据各点间的空间距离
3. 将样本点按距离由近到远排序
4. 挑选前 $k$ 个样本数据点，作为决策基础
5. 将 $k$ 个样本所属类别中多数类作为预测类

算法的核心（需要调参的）是 $k$ 的具体数量和距离的测算方法（欧氏距离、曼哈顿距离等）；由于需要一定的存储空间保存训练数据，无法给出模型结构，计算耗时长等问题限制了knn的使用范围。

### 3.2.2 朴素贝叶斯
朴素贝叶斯是基于概率论的分类方法，其核心是**条件概率**、**全概率公式**和**贝叶斯准则**推导的，原理如下：
- 条件概率：$P(X|Y) = \frac{P(XY)}{P(Y)}$
- 贝叶斯准则：$P(Y|X) = \frac{P(X|Y) * P(Y)}{P(X)}$
- 全概率公式：$P(X) = \sum_{i=1}^n P(X|y_i)P(y_i)$，其中 $P(y_i)$ 互斥且 $\sum_{i=1}^n P(y_i) = 1$
- $P(Y=y_k|X) = \frac{P(X|y_k) * P(y_k)}{\sum_{i=1}^n P(X|y_i)P(y_i)}$，其中 $n$ 代表 $Y$ 可以取值的类别，$y_k$表示其中某个类别
- $P(Y=y_k|X)$ 中的计算的难点是 $P(X|y_k)$，朴素贝叶斯假设特征之间相互独立（很难满足，牺牲准确性换取计算简化），则 $P(X|y_k) = P(x_1|y_k) * P(x_2|y_k) * …… * P(x_n|y_k)$

上述一系列公式中，$P(x_i|y_k)$ 潜藏着如下缺陷：
1. 局部值为零，影响最终预测结果
2. 局部值极小，最终结果趋于零，且计算机在计算时会溢出直接输出零值

针对上述问题分别引申出如下解决方案：
- **拉普拉斯变换**：![](https://static.sitestack.cn/projects/Vay-keen-Machine-learning-learning-notes/707144452a627e2666e889c22dc90fa6.png)
- 对数计算：$ P(x_1|y_k) * P(x_2|y_k) * …… * P(x_n|y_k) = e^{ln(P(x_1|y_k))+ln(P(x_2|y_k))+……+ln(P(x_n|y_k))} $

此外，根据朴素贝叶斯内层概率内核不同划分为：
- 伯努利实验：注重0-无/1-有，不考虑次数权重，常见的是词集模型
- 多项式实验：考虑次数权重，常见的是词袋模型

### 3.2.3 决策树
决策树是一个具有高度可解释性的模型，其内核是基于特征分割数据，每次通过对数据的分割，将原始数据划分为单一类别占比更高的子集，直到子集中分类近乎全为同一类别（后面我们称为纯净度），主流的算法有ID3、C4.5及CART。

### 3.2.3.1 ID3 & C4.5
ID3和C4.5都是基于信息论进行不纯度度量的且C4.5是ID3的改进方法，故先讲解ID3算法。回顾前面损失函数提及的信息熵：$ \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $，当 $p_i = 0.5$ 时，信息熵取最大值；当 $p_i = 1$ 时，信息熵取最小值0。那么信息熵是可以作为度量数据集合纯净度的指标，而ID3就是使用的此指标。ID3如何选择哪个分割特征并且基于该特征的哪些阈值划分数据呢？那必定是希望数据划分后的整体纯净度更高，ID3中使用信息增益来刻画划分前后信息熵的变化，对于节点 $k$ 与下层 $k+1$ 节点集的信息增益公式为：$ -\sum_{i=1}^n p_{(k)i}log_2p_{(k)i} - (-\sum\frac{N_{(k+1)l}}{N_{kl}} \sum_{j=1}^m p_{(k+1)j}log_2p_{(k+1)j}) $，这个公式我表述的并不准确大意是用原始的信息熵减去分支节点信息熵的加权和作为信息增益，改值越大，说明特征划分后的数据越纯净；然后不断的循环直至当前节点无法划分或特征全部划分完毕。


现在我们将要开始进行构建ID3决策树了！

ID3分割数据使用的是信息增益

数据的分割
类别占比的纯净度
信息


没有剪枝策略，容易过拟合；
信息增益准则对可取值数目较多的特征有所偏好，类似“编号”的特征其信息增益接近于 1；
只能用于处理离散分布的特征；
没有考虑缺失值

相对于 ID3 的缺点对应有以下改进方式：
引入悲观剪枝策略进行后剪枝；
引入信息增益率作为划分标准；
将连续特征离散化，假设 n 个样本的连续特征 A 有 m 个取值，C4.5 将其排序并取相邻两样本值的平均数共 m-1 个划分点，分别计算以该划分点作为二元分类点时的信息增益，并选择信息增益最大的点作为该连续特征的二元离散分类点；
对于缺失值的处理可以分为两个子问题：
问题一：在特征值缺失的情况下进行划分特征的选择？（即如何计算特征的信息增益率）
问题二：选定该划分特征，对于缺失该特征值的样本如何处理？（即到底把这个样本划分到哪个结点里）
针对问题一，C4.5 的做法是：对于具有缺失值特征，用没有缺失的样本子集所占比重来折算；
针对问题二，C4.5 的做法是：将样本同时划分到所有子节点，不过要调整样本的权重值，其实也就是以不同概率划分到不同节点中。

### 3.2.2.2 CART


### 3.2.4 支持向量机
支持向量
SVM模型有两个非常重要的参数C与gamma。其中 C是惩罚系数，即对误差的宽容度。c越高，说明越不能容忍出现误差,容易过拟合。C越小，容易欠拟合。C过大或过小，泛化能力变差
gamma是选择RBF函数作为kernel后，该函数自带的一个参数。隐含地决定了数据映射到新的特征空间后的分布，gamma越大，支持向量越少，gamma值越小，支持向量越多。支持向量的个数影响训练与预测的速度。

## 3.3 评估模型

## 3.4 交叉验证
模型调参神器

---
参考资料：
- [Sklearn 与 TensorFlow 机器学习实用指南](https://www.bookstack.cn/books/hands_on_Ml_with_Sklearn_and_TF)
- [最小二乘法](https://zhuanlan.zhihu.com/p/38128785)
- [极大似然估计与最大后验概率估计](https://zhuanlan.zhihu.com/p/40024110)
- [信息论](https://zhuanlan.zhihu.com/p/36385989)
- [理解朴素贝叶斯分类的拉普拉斯平滑](https://zhuanlan.zhihu.com/p/26329951)
- [如何通俗的解释交叉熵与相对熵](https://www.zhihu.com/question/41252833)
- [波士顿房价预测任务](https://www.paddlepaddle.org.cn/tutorials/projectdetail/1515038#anchor-13)